Description:   
This script should load the iJO1366 and iHK1487 models from the BiGG
database, make modifications so that they reperesent the strains used in
the Liu et al study and then output them to the 'stable' directory
with the names 'AG.xml' and 'GD.xml' respectively.

Directionality of exchange reactions: https://groups.google.com/g/cobra-toolbox/c/JE7LMkkqlT0/m/uz2oCaQEEwAJ

##### Agenda:
- Load existing 'parent' models from BiGG
- Construct 'unmodified' strain representations as used in the paper, by editing models to spec.
- Optimise each with default medium conditions & generate summary to establish baseline.
    - Just use the provided representation of minimal media 
- Make strain modifications per paper (Liu) and simulate in default medium.
    - We are not expecting viability at this stage as they have been made auxotrophic for either tyrosine or phenylalanine.
- Adjust media to facilitate their respective auxotrophies and obtain optimization summaries.


In [1]:
import cobra
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
from cobra import Model, Reaction, Metabolite
from cameo import models
from cameo import load_model

# Start: AG Strain Reconstruction

In [2]:
# Load base model for AG strain and save locally 
# m = load_model('iJO1366')
# cobra.io.write_sbml_model(m,"../Stable/iJO1366.xml")
# m = load_model("../Stable/iJO1366.xml")
m = load_model("../../data/iJO1366.xml")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-10


## iJ0166 -> BW25113
Reconstructing the starting strain used in the Liu et al paper

In [3]:
# Gene Knockout targets:
# araBAD, rhaBAD, and lacZ
# Gene Id's:
# araA b0062
# araB b0063
# araD   -
# rhaA b3903
# rhaB b3904
# rhaD b3902
# lacZ b0344

# Target Genes 
t = [
    "araA", "araB", "araD"
    "rhaD","rhaA","rhaB",
    "lacZ"
    ]

# Identify affected reactions:
# for g in m.genes:
#     if g.name in t:
#         for r in g.reactions:
#             print(r.id)

# Resultant list: 
t_r = ["ARAI","RMI","LYXI","RBK_L1","XYLK","XYLK2","RMK","LACZ"]

for r in m.reactions:
    if r.id in t_r:
        print(r.id, r.bounds)


ARAI (-1000.0, 1000.0)
LACZ (0.0, 1000.0)
LYXI (0.0, 1000.0)
RBK_L1 (0.0, 1000.0)
XYLK (0.0, 1000.0)
XYLK2 (0.0, 1000.0)
RMI (-1000.0, 1000.0)
RMK (0.0, 1000.0)


In [4]:
## Apply first set of knockouts
tgts = [g for g in m.genes if g.name in t]

for g in tgts:
    g.knock_out()

# See if KO has had an effect:
for r in m.reactions:
    if r.id in t_r:
        print(r.id, r.bounds)

ARAI (0, 0)
LACZ (0, 0)
LYXI (0, 0)
RBK_L1 (0, 0)
XYLK (0.0, 1000.0)
XYLK2 (0.0, 1000.0)
RMI (0, 0)
RMK (0, 0)


In [5]:
m.slim_optimize()


0.9823718127269814

##  BW25113 -> AG

Subsequent modifications made in Liu et al to create the phenylalanine  
auxotrophic AG strain.  
List of modification made:  
1. Insertion	synkdc4 	decarboxylase reaction		4HPP -> 4HPAA  
2. Deletion	ΔptsG	Xylose preference		Glu[e] -> Glu[AG_c]  
3. Deletion	ΔpykA			PEP -> PYR  
4. Deletion	ΔpykF			PEP -> PYR  
5. Deletion	ΔtyrR 			E4P -> CHA  
6. Deletion	ΔpheA			CHA -> PHE  
7. Overexpression	aroGfbr-tyrAfbr-aroE			(Uncertain how to implement)  
8. Deletion	ΔfeaB	Preventing improves tyrosol yield directly.		4HPAA -> 4HPA  
9. Deletion	ΔmanZ	Prevents glucose uptake		  

Additonally this requires the implementation of:  
- ADH alcohol dehydrogenase reaction: HPAA -> 4-tyrosol  
    - 4 tyrosol is a new metabolite   
- Tryosol export reaction  
    - TODO: How is this manufactured in situ?   
- Phenylalanine uptake reaction  

#### synkdc Insertion

Context:  
4HPP is identified as an intermediate in the synthetic pathway for tyrosol.  
“Thus tyrosol biosynthetic pathway would be assembled in E. coli from tyrosine via 4HPP,  
which generates 4-hydroxylphenylacetaldehyde (4HPAA) by decarboxylation,   
and further turns into 4-hydroxyphenylethanol, i.e., tyrosol, by reduction reaction.” (Liu et al., 2018, p. 245)  
2-keto acid decarboxylase (KDC) Aro10 gene from S.cervisae was identified to catalyse this reaction,  
so the researchers searched BLAST to find homologous sequences.  
They identified two more to add to aro10 and synthesized all three, finding that synkdc4 was most efficacious -fig .
synkdc4 is a codon-optimised version of the kdc4 from Pichia pastoris GS115 (model: ihGlycopastoris_COBRA?) TODO check kdc4 reactions 

Given reaction:  
4HPP -> 4HPAA
4-hydroxyphenylpyruvate; 4-hydroxyphenylacetaldehyde  
Metabolite ID's 
3,4HPP 34hpp_c  3-(4-Hydroxyphenyl)pyruvate  
4HPP: ?? (cytoplasmic)   
4HPAA: 4hoxpacd_X (periplasmic/extracellular)  

Check if these metabolites are present.  
for i in JO.metabolites:  
    if "ydroxyphen" in i.name:  
        print(i.id, i.name)  
        
Can 34hpp_c be treated as HPP?  
https://pubchem.ncbi.nlm.nih.gov/compound/3-_4-Hydroxyphenyl_pyruvate#section=Canonical-SMILES

Think it is reasonable to treat 34hpp_c or vice versa, may well be a protonation situation.  
In any case the following reaction achieves the effect:  
3-(4-hydroxyphenyl)pyruvate + H+ → (4-hydroxyphenyl)acetaldehyde_c + CO2
Source: https://metacyc.org/META/NEW-IMAGE?type=REACTION&object=4.1.1.80-RXN

In [6]:
### Create Metabolites, Reaction & insert into model:
# Access existing metabolites to add to the reaction:
HPP = [i for i in m.metabolites if i.id == "34hpp_c"][0] # Acesses 34hpp_c using list as naming convention is not compatible.
# Reactions associated with 34hpp_c: 
# print("Reactions associated with 34hpp_c")
# print(m.reactions.PPND.name) # Prephenate dehydrogenase
# print(m.reactions.PPND.reaction) # Prephenate dehydrogenase
# print(m.reactions.TYRTA.name) # Tyrosine transaminase <- 
# print(m.reactions.TYRTA.reaction) # Tyrosine transaminase

# Hydrogen
H_c = m.metabolites.h_c
# Carbon Dioxide
CO2_c = m.metabolites.co2_c

# Creation of 'synkdc4' as a reaction called KDC4
# Ref: https://cobrapy.readthedocs.io/en/latest/building_model.html#Model,-Reactions-and-Metabolites
# New Metabolites:
# 1. HPAA_c : Addiing HPAA to the cytosol, maintaining the naming convention used across other compartments:4hoxpacd_x
HPAA_c = Metabolite(
    '4hoxpacd_c',
    formula='C8H8O2',
    name='4-hydroxyphenylacetaldehyde',
    compartment='c')

# Creation of KDC4 Reaction 
reaction = Reaction('KDC4')
reaction.name = '4-hydroxyphenylpyruvate decarboxylation'
reaction.subsystem = 'Synthetic Tryosol Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

# Add metabolite to reaction 
# Have to add metabolite objects..
reaction.add_metabolites({
    HPP: -1.0,
    H_c: -1.0,
    HPAA_c: 1.0,
    CO2_c: 1.0
})

m.add_reactions([reaction]) # Add KDC4 to model; not associated with any genes.

# Check added reaction
m.reactions.KDC4

m.reactions.KDC4.check_mass_balance() # Balanced reactions return empty dict

{}

In [7]:
for met in m.metabolites:
    if "hoxpacd" in met.id:
        print(met)

4hoxpacd_e
4hoxpacd_p
4hoxpacd_c


#### Tyrosol ADH Synthesis
Add metabolite/reaction for Alcohol dehydrogenase synthesis of Tyrosol


In [8]:
TYROSOL_c = Metabolite(
    'Tyrosol_c',
    formula='C8H10O2',
    name='4-hydroxyphenylethanol (4-Tyrosol)',
    compartment='c')

NADH_c = m.metabolites.nadh_c
NAD_c = m.metabolites.nad_c

reaction = Reaction('HPAA_ADH')
reaction.name = '4-hydroxyphenylacetylaldehyde dehydrogenase '
reaction.subsystem = 'Synthetic Tryosol Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

# Add metabolite to reaction 
# Have to add metabolite objects..
reaction.add_metabolites({
    HPAA_c: -1.0,
    NADH_c: -1.0,
    H_c: -1.0,
    TYROSOL_c: 1.0,
    NAD_c: 1.0
})

m.add_reactions([reaction])



#### Tyrosol Transport Reaction
Add extracellular compartment for tyrosol and create exchange reaxction to allow secretion needed for cross feeding

In [9]:
# Create extracellular metabolite for conversion into.
TYROSOL_e = Metabolite(
    'Tyrosol_e',
    formula='C8H10O2',
    name='4-hydroxyphenylethanol (4-Tyrosol)',
    compartment='e')

reaction = Reaction('Tyrosolex')
reaction.name = 'Tyrosol transport (cytoplasm to extracellular)'
reaction.subsystem = 'Synthetic Tryosol Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

# Add metabolite to reaction 
# Have to add metabolite objects..
reaction.add_metabolites({
    TYROSOL_c: -1.0,
    TYROSOL_e: 1.0
})

m.add_reactions([reaction])

# Critically important to add the export reaction as well for the steady state assumption
reaction = Reaction("EX_Tyrosol_e")
reaction.name = "Tyrosol Exchange Reaction"
reaction.subsystem = "Synthetic Tyrosol Production"
reaction.lower_bound = 0
reaction.upper_bound = 1000
reaction.add_metabolites({
    TYROSOL_e: -1.0
})
m.add_reactions([reaction])

#### Allow Xylose Uptake By Default
This provides carbon source when ability to uptake glucose is knocked out below.

In [10]:
for i in m.exchanges:
    if "ylos" in i.name:
        print(i.name, i.id)

m.reactions.EX_xyl__D_e
m.reactions.EX_xyl__D_e.lower_bound = -10.0

D-Xylose exchange EX_xyl__D_e


#### Gene Knockouts

In [11]:
t = [
    "ptsG",
    "pykA",
    "pykF",
    "tyrR",
    "pheA",
    "feaB",
    "manZ"
]

print([(i.name,i.id) for i in m.genes if i.name in t]) #  tyrR is missing from the model TODO
### Identifying genes/reactions that may represent those reactions indicated by tyrR and manZ
# 1. Deletion	ΔtyrR 			?
# tyrR
# "pathway-specific transcriptional regulatory
# gene tyrR led to enhanced expression of genes relevant to L-
# tyrosine biosynthesis, including aroG, tyrB and aroL" - yao 2013 
# This indicates it suppresses production of tyrosine - may not be accounted for properly in the model.
# Action: ignore 


# Identify affected reactions:
# for g in m.genes:
#     if g.name in t:
#         for r in g.reactions:
#             print(r.id)

# Resultant list: 
t_r = ["PPNDH","CHORM","GLCptspp","ACGAptspp","ALDD19xr","ACMANAptspp","GLCptspp","FRUpts2pp","GAMptspp","MANptspp","PYK","PYK"]

print("Before")
for r in m.reactions:
    if r.id in t_r:
        print(r.id, r.bounds)

tgts = [g for g in m.genes if g.name in t]


# for i in range(len(tgts)):
#     # tgts = [g for g in model.genes if g.name in t]
#     with m as model:
#         tgts[i].knock_out()
#         print(tgts[i], model.slim_optimize())
    

for g in tgts:
    g.knock_out()
# See if KO has had an effect:
print("\nAfter:")
for r in m.reactions:
    if r.id in t_r:
        print(r.id, r.bounds)


[('pheA', 'b2599'), ('ptsG', 'b1101'), ('feaB', 'b1385'), ('manZ', 'b1819'), ('pykF', 'b1676'), ('pykA', 'b1854')]
Before
ACGAptspp (0.0, 1000.0)
ACMANAptspp (0.0, 1000.0)
ALDD19xr (-1000.0, 1000.0)
CHORM (0.0, 1000.0)
FRUpts2pp (0.0, 1000.0)
GLCptspp (0.0, 1000.0)
GAMptspp (0.0, 1000.0)
MANptspp (0.0, 1000.0)
PPNDH (0.0, 1000.0)
PYK (0.0, 1000.0)

After:
ACGAptspp (0.0, 1000.0)
ACMANAptspp (0, 0)
ALDD19xr (0, 0)
CHORM (0.0, 1000.0)
FRUpts2pp (0, 0)
GLCptspp (0.0, 1000.0)
GAMptspp (0, 0)
MANptspp (0, 0)
PPNDH (0, 0)
PYK (0, 0)


#### Manually prevent uptake of glucose
For the model the KO of ptsG and manZ are insufficient   to prevent glucose uptake from media so this is enforced by manually constriaing the two GLCtex transport reactions 

In [12]:
# Literally what happened to this?
m.reactions.GLCtex_copy1.bounds = (0,0)
m.reactions.GLCtex_copy2.bounds = (0,0)

#### Add Phenylalanine To Media
Else will become non-viable as soon as pheA gene is KO.

In [13]:
# At this point it should not be able to grow ubless phenylalanine is externally avialible.
print(m.slim_optimize())

m.reactions.EX_phe__L_e.bounds = (-3,10)
# Should now be able to grow
print(m.slim_optimize())


0.0
0.8182241571069285


Tests

In [14]:
# AG should not be able to uptake glucose
 
# Check the effect of glucose on the medium
with m as model:
    # model.reactions.EX_xyl__D_e.bounds = (0,0)
    model.reactions.EX_glc__D_e.bounds = (0,0)
    print(model.slim_optimize())
    model.reactions.EX_glc__D_e.bounds = (-10,100)
    print(model.slim_optimize())



0.8182241571069264
0.8182241571069264


In [15]:
# check if can grow without xylose
# m.medium
with m as model:
    model.reactions.EX_xyl__D_e.bounds = (0,0)
    print(model.slim_optimize())
    model.reactions.EX_xyl__D_e.bounds = (-10,100)
    print(model.slim_optimize())

nan
0.8182241571069291


In [16]:
# Final media representaion for single culture growth.
m.medium

{'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_phe__L_e': 3,
 'EX_cl_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_xyl__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0}

#### aroGfbr-tyrAfbr-aroE Overexpression
Unsure how to implement

#### Save Model

In [17]:
cobra.io.write_sbml_model(m,"../../data/AG.xml")

# END: AG Strain Reconstruction

In [18]:
# Clear Jupyter notebook variables ad reload required libraries
%reset -f

In [19]:
import cobra
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
from cobra import Model, Reaction, Metabolite
from cameo import models
from cameo import load_model

# START: GD Strain Reconstruction

Reminder: iHK1487 is the most up to date representation of BL21(DE3)  
**Modifications**:  
OverExpression	pgm	Phosphoglucomutase		G6P ⇌ G1P  
OverExpression	galU	UTP-glucose-1-phosphate uridylyltransferase 		g1p_c + h_c + utp_c → ppi_c + udpg_c  
Deletion	ushA			  
Deletion	tyrA			  
Deletion	xylA	Xylose Isomerase  
Insertion   synugt85a1  ??	

In [20]:
# m = load_model("../Stable/iHK1487.xml")
m = load_model("../../data/iHK1487.xml")

#### Gene Deletions
Cross-Identified from iJO1366:  
name  x-ref   id  
b2600 tyrA ECD_02489   
b0480 ushA ECD_00431   
b3565 xylA ECD_03417  

In [21]:
m

Name,iHK1487
Memory address,0x014a7f9b3d90
Number of metabolites,1877
Number of reactions,2701
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iHK1487_core - 1.0*BIOMASS_Ec_iHK1487_core_reverse_c9aec
Compartments,"cytosol, extracellular space, periplasm"


In [22]:
t = ["ECD_02489","ECD_00431","ECD_03417"]
tgts = [i for i in m.genes if i.id in t]
# for g in tgts:
#     print(g.reactions)


print("Before:")
for g in tgts:
    for r in g.reactions:
        print(r.id, r.bounds)

for g in tgts:
    g.knock_out()

print("\nAfter:")
for g in tgts:
    for r in g.reactions:
        print(r.id, r.bounds)


Before:
CHORM (0.0, 1000.0)
PPND (0.0, 1000.0)
XYLI1 (-1000.0, 1000.0)
XYLI2 (-1000.0, 1000.0)
UGLCURPpp (0.0, 1000.0)
NTD10pp (0.0, 1000.0)
NTD8pp (0.0, 1000.0)
NTD12pp (0.0, 1000.0)
UACGAMPpp (0.0, 1000.0)
NTD5pp (0.0, 1000.0)
UDPGALPpp (0.0, 1000.0)
NTD6pp (0.0, 1000.0)
NTD4pp (0.0, 1000.0)
UACGALPpp (0.0, 1000.0)
NTD1pp (0.0, 1000.0)
NTD2pp (0.0, 1000.0)
NTD11pp (0.0, 1000.0)
NTD9pp (0.0, 1000.0)
NTD3pp (0.0, 1000.0)
NTD7pp (0.0, 1000.0)
UDPGPpp (0.0, 1000.0)

After:
CHORM (0.0, 1000.0)
PPND (0, 0)
XYLI1 (0, 0)
XYLI2 (0, 0)
UGLCURPpp (0, 0)
NTD10pp (0.0, 1000.0)
NTD8pp (0.0, 1000.0)
NTD12pp (0.0, 1000.0)
UACGAMPpp (0, 0)
NTD5pp (0.0, 1000.0)
UDPGALPpp (0, 0)
NTD6pp (0.0, 1000.0)
NTD4pp (0.0, 1000.0)
UACGALPpp (0, 0)
NTD1pp (0.0, 1000.0)
NTD2pp (0.0, 1000.0)
NTD11pp (0.0, 1000.0)
NTD9pp (0.0, 1000.0)
NTD3pp (0.0, 1000.0)
NTD7pp (0.0, 1000.0)
UDPGPpp (0, 0)


#### Allow Phenylalanine Export
pathway exists natively

#### Add Tyrosol to Model & Allow Uptake

In [23]:
# Create tyrosol metabolites
TYROSOL_e = Metabolite(
    'Tyrosol_e',
    formula='C8H10O2',
    name='4-hydroxyphenylethanol (4-Tyrosol)',
    compartment='e')

TYROSOL_c = Metabolite(
    'Tyrosol_c',
    formula='C8H10O2',
    name='4-hydroxyphenylethanol (4-Tyrosol)',
    compartment='c')

reaction = Reaction('Tyrosolex')
reaction.name = 'Tyrosol transport (extracellular to cytoplasm)'
reaction.subsystem = 'Synthetic Salidroside Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

# Add metabolite to reaction 
# Have to add metabolite objects..
reaction.add_metabolites({
    TYROSOL_e: -1.0,
    TYROSOL_c: 1.0
})

m.add_reactions([reaction])

# Check added reaction
# BL21.reactions.{}
# BL21.reactions.{}.check_mass_balance()


# Add in exchange reaction as well for monoculture testing
reaction = Reaction('EX_Tyrosol_e')
reaction.name = 'Tyrosol Exchange'
reaction.subsystem = 'Synthetic Salidroside Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

reaction.add_metabolites({
    TYROSOL_e: -1.0
})

m.add_reactions([reaction])

#### Allow Tyrosine uptake from media

In [24]:
m.reactions.EX_tyr__L_e.lower_bound = -3.0 # Allow import of Tyrosine
m.reactions.EX_tyr__L_e

Reaction identifier,EX_tyr__L_e
Name,L-Tyrosine exchange
Memory address,0x014a08396aa0
Stoichiometry,tyr__L_e <=> L-Tyrosine <=>
GPR,
Lower bound,-3.0
Upper bound,1000.0


#### Salisdroside BioSynthesis

In [25]:
# synugt85a1 insertion 
# Salidroside Production
# https://metacyc.org/compound?orgid=META&id=CPD-13354#tab=RXNS
# Reaction:
# UDP-α-D-glucose + 4-tyrosol → salidroside + UDP + H+

# UDP GLucose is a synonym
# UDPGlucose formula: C15H22N2O17P2
# m.metabolites.udpg_c # Matches

for i in m.metabolites:
    if "lucose" in i.name:
        print(i.id," ::: ",i.name)

H_c = m.metabolites.h_c
UDPGLC_c = m.metabolites.udpg_c
UDP_c = m.metabolites.udp_c

# Create Salidroside Metabolites
Salidroside_c = Metabolite(
    'Salidroside_c',
    formula='C14H20O7',
    name='8-O-b-D-glucoside of tyrosol (salidroside)',
    compartment='c')

Salidroside_e = Metabolite(
    'Salidroside_e',
    formula='C14H20O7',
    name='8-O-b-D-glucoside of tyrosol (salidroside)',
    compartment='e')


reaction = Reaction('SALIDROSIDE_BIOSYNTHESIS')
reaction.name = 'SALIDROSIDE_BIOSYNTHESIS'
reaction.subsystem = 'Synthetic Salidroside Production'
reaction.lower_bound = 0  # This is the default
reaction.upper_bound = 1000  # This is the default 

# Add metabolite to reaction 
# Have to add metabolite objects..
# UDP-α-D-glucose + 4-tyrosol → salidroside + UDP + H+
# Bypass salidroside_c -> _e reaction for expediancy
reaction.add_metabolites({
    TYROSOL_c: -1.0,
    UDPGLC_c: -1.0,
    Salidroside_e: 1.0,
    UDP_c: 1.0,
    H_c: 1.0
})

m.add_reactions([reaction])



6pgg_c  :::  6-Phospho-beta-D-glucosyl-(1,4)-D-glucose
dtdp4d6dg_c  :::  DTDP-4-dehydro-6-deoxy-D-glucose
g6p_e  :::  D-Glucose 6-phosphate
g1p_e  :::  D-Glucose 1-phosphate
udcpgl_c  :::  Undecaprenol-diphosphate-glucose
g6p_p  :::  D-Glucose 6-phosphate
2doxg6p_c  :::  2 Deoxy D glucose 6 phosphate C6H11O8P
udpg_e  :::  UDPglucose
adpglc_c  :::  ADPglucose C16H23N5O15P2
glc__D_c  :::  D-Glucose
g1p_p  :::  D-Glucose 1-phosphate
udpg_c  :::  UDPglucose
g1p_c  :::  D-Glucose 1-phosphate
udpg_p  :::  UDPglucose
acglc__D_c  :::  6-Acetyl-D-glucose
glc__D_e  :::  D-Glucose
g6p_c  :::  D-Glucose 6-phosphate
glc__D_p  :::  D-Glucose
dtdpglu_c  :::  DTDPglucose
2dglc_p  :::  2 Deoxy D glucose C6H12O5
2dglc_c  :::  2 Deoxy D glucose C6H12O5
2dglc_e  :::  2 Deoxy D glucose C6H12O5


In [26]:
# Open exchange reaction
m.reactions.EX_co2_e

reaction = Reaction('EX_Salidroside_e')
reaction.name = 'Salidroside Exchange'
reaction.subsystem = 'Synthetic Salidroside Production'
reaction.lower_bound = -0  # This is the default
reaction.upper_bound = 1000  # This is the default 

reaction.add_metabolites({
    Salidroside_e: -1.0
})

m.add_reactions([reaction])

In [27]:
# Check added reaction
m.reactions.EX_Salidroside_e
m.reactions.EX_Salidroside_e.check_mass_balance()

{'C': -14.0, 'H': -20.0, 'O': -7.0}

# END: GD Strain Reconstruction

In [28]:
m.reactions.EX_tyr__L_e.bounds = (0,10)
print(m.slim_optimize())

m.reactions.EX_tyr__L_e.bounds = (-3,10)
# Should now be able to grow
print(m.slim_optimize())

0.0
0.8294303202662239


In [29]:
m.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_tyr__L_e': 3,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 18.5,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [30]:
write_sbml_model(m,"../../data/GD.xml")

# Done